In [66]:
import re
import pandas as pd

from pathlib import Path

In [52]:
pd.set_option('display.max_colwidth', None)

In [53]:
REPORT_DIR = '/Users/artemvopilov/Documents/Ozon'
REPORT_NAME = 'report_goods_20231201_20240114.xlsx'

### Read data

In [54]:
report_path = Path(REPORT_DIR, REPORT_NAME)

In [55]:
report_df = pd.read_excel(report_path, engine='openpyxl', sheet_name='Начисления', parse_dates=True)

/Users/artemvopilov/Business/ozon_analytics/.venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Describe

In [56]:
report_df.columns

Index(['Дата начисления', 'Тип начисления',
       'Номер отправления или идентификатор услуги',
       'Дата принятия заказа в обработку или оказания услуги',
       'Склад отгрузки', 'SKU', 'Артикул', 'Название товара или услуги',
       'Количество', 'За продажу или возврат до вычета комиссий и услуг',
       'Ставка комиссии', 'Комиссия за продажу', 'Сборка заказа',
       'Обработка отправления (Drop-off/Pick-up) (разбивается по товарам пропорционально количеству в отправлении)',
       'Магистраль',
       'Последняя миля (разбивается по товарам пропорционально доле цены товара в сумме отправления)',
       'Обратная магистраль', 'Обработка возврата',
       'Обработка отмененного или невостребованного товара (разбивается по товарам в отправлении в одинаковой пропорции)',
       'Обработка невыкупленного товара', 'Логистика', 'Индекс локализации',
       'Обратная логистика', 'Итого'],
      dtype='object')

In [57]:
report_df.dtypes

Дата начисления                                                                                                     datetime64[ns]
Тип начисления                                                                                                              object
Номер отправления или идентификатор услуги                                                                                  object
Дата принятия заказа в обработку или оказания услуги                                                                datetime64[ns]
Склад отгрузки                                                                                                              object
SKU                                                                                                                        float64
Артикул                                                                                                                    float64
Название товара или услуги                                                         

In [58]:
report_df.head()

,Дата начисления,Тип начисления,Номер отправления или идентификатор услуги,Дата принятия заказа в обработку или оказания услуги,Склад отгрузки,SKU,Артикул,Название товара или услуги,Количество,За продажу или возврат до вычета комиссий и услуг,...,Магистраль,Последняя миля (разбивается по товарам пропорционально доле цены товара в сумме отправления),Обратная магистраль,Обработка возврата,Обработка отмененного или невостребованного товара (разбивается по товарам в отправлении в одинаковой пропорции),Обработка невыкупленного товара,Логистика,Индекс локализации,Обратная логистика,Итого
0,2023-12-01,"Доставка и обработка возврата, отмены, невыкупа",46593954-0044-1,2023-11-20 08:29:30,FBO,1.211572e+09,9004.0,"Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.",1,0,...,0,0.00,0,0,0,0,-63.8,62.0,0,-63.80
1,2023-12-01,"Доставка и обработка возврата, отмены, невыкупа",46593954-0044-1,2023-11-20 08:29:30,FBO,1.211572e+09,9004.0,"Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.",1,0,...,0,0.00,0,-15,0,-50,0.0,NaN,-58,-123.00
2,2023-12-01,"Доставка и обработка возврата, отмены, невыкупа",23545563-0150-9,2023-11-15 00:15:18,FBO,1.211572e+09,9003.0,"Зарядное устройство для iPhone, iPad, AirPods. Адаптер 20W, кабель Lightning to USB Type-C, 1 метр белый. Быстрая зарядка.",1,0,...,0,0.00,0,-65,0,0,0.0,NaN,-58,-123.00
3,2023-12-01,Доставка покупателю,51086215-0024-1,2023-11-12 02:42:38,FBO,1.211572e+09,9003.0,"Зарядное устройство для iPhone, iPad, AirPods. Адаптер 20W, кабель Lightning to USB Type-C, 1 метр белый. Быстрая зарядка.",1,478,...,0,-7.03,0,0,0,0,-63.8,61.0,0,318.74
4,2023-12-01,Доставка покупателю,80849669-0112-2,2023-11-29 14:56:22,FBO,1.211572e+09,9003.0,"Зарядное устройство для iPhone, iPad, AirPods. Адаптер 20W, кабель Lightning to USB Type-C, 1 метр белый. Быстрая зарядка.",1,672,...,0,-31.80,0,0,0,0,-63.8,62.0,0,452.08


In [59]:
report_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 24 columns):
 #   Column                                                                                                            Non-Null Count  Dtype         
---  ------                                                                                                            --------------  -----         
 0   Дата начисления                                                                                                   196 non-null    datetime64[ns]
 1   Тип начисления                                                                                                    196 non-null    object        
 2   Номер отправления или идентификатор услуги                                                                        167 non-null    object        
 3   Дата принятия заказа в обработку или оказания услуги                                                              196 non-null    datetime64

In [60]:
report_df.describe(include='all')

,Дата начисления,Тип начисления,Номер отправления или идентификатор услуги,Дата принятия заказа в обработку или оказания услуги,Склад отгрузки,SKU,Артикул,Название товара или услуги,Количество,За продажу или возврат до вычета комиссий и услуг,...,Магистраль,Последняя миля (разбивается по товарам пропорционально доле цены товара в сумме отправления),Обратная магистраль,Обработка возврата,Обработка отмененного или невостребованного товара (разбивается по товарам в отправлении в одинаковой пропорции),Обработка невыкупленного товара,Логистика,Индекс локализации,Обратная логистика,Итого
count,196,196,167,196,84,1.660000e+02,166.000000,188,196.000000,196.000000,...,196.0,196.000000,196.0,196.000000,196.0,196.000000,196.000000,82.000000,196.000000,196.000000
unique,NaN,7,159,NaN,1,NaN,NaN,32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Оплата эквайринга,46593954-0044-1,NaN,FBO,NaN,NaN,"Держатель кухонный для разделочных досок, для сковород, тарелок, кастрюль, 3 см х 5 см х 5 см, 1 шт",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,82,2,NaN,84,NaN,NaN,49,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2023-12-23 00:14:41.632653056,NaN,NaN,2023-12-21 01:28:19.214285568,NaN,1.260451e+09,9008.644578,NaN,0.933673,145.801020,...,0.0,-6.919898,0.0,-0.408163,0.0,-0.255102,-28.986224,53.121951,-0.591837,63.065918
min,2023-12-01 00:00:00,NaN,NaN,2023-11-12 02:42:38,NaN,1.201776e+09,9001.000000,NaN,0.000000,0.000000,...,0.0,-43.750000,0.0,-65.000000,0.0,-50.000000,-226.800000,20.000000,-58.000000,-550.000000
25%,2023-12-10 00:00:00,NaN,NaN,2023-12-09 15:01:34,NaN,1.201776e+09,9001.000000,NaN,1.000000,0.000000,...,0.0,-8.360000,0.0,0.000000,0.0,0.000000,-63.800000,38.000000,0.000000,-8.380000
50%,2023-12-19 00:00:00,NaN,NaN,2023-12-17 20:04:21.500000,NaN,1.297355e+09,9005.000000,NaN,1.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,60.000000,0.000000,-1.810000
75%,2024-01-06 06:00:00,NaN,NaN,2024-01-05 17:54:08.500000,NaN,1.309528e+09,9011.000000,NaN,1.000000,152.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,62.000000,0.000000,53.092500
max,2024-01-14 00:00:00,NaN,NaN,2024-01-13 11:39:58,NaN,1.325099e+09,9101.000000,NaN,5.000000,1012.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,77.000000,0.000000,720.610000


### Statistics

##### Turnover

In [139]:
report_df['Итого'].abs().sum()

21545.899999999998

##### Income

In [140]:
report_df['Итого'].sum()

12360.919999999998

##### Income by operation type

In [141]:
report_df.groupby('Тип начисления', as_index=False).agg(Sum=('Итого', 'sum'))

,Тип начисления,Sum
0,"Доставка и обработка возврата, отмены, невыкупа",-309.80
1,Доставка покупателю,16452.17
2,Обработка товара в составе грузоместа на FBO,0.00
3,Оплата эквайринга,-272.95
4,Приобретение отзывов на платформе,-1512.00
5,Услуги продвижения товаров,-1921.50
6,Утилизация,-75.00


##### Income by operation type and product

In [155]:
# report_df\
report_df[
    (report_df['Тип начисления'] != 'Доставка покупателю') 
    | (report_df['За продажу или возврат до вычета комиссий и услуг'] > 100)
]\
    .replace(to_replace='^.*в формате "(.+?)" в период.*$', value=r'\1', regex=True)\
    .groupby(['Тип начисления', 'Название товара или услуги'], as_index=False, dropna=False)\
    .agg(Sum=('Итого', 'sum'), Count=('Количество', 'sum'))\
    .assign(Avg=lambda row: row['Sum'] / row['Count'])

,Тип начисления,Название товара или услуги,Sum,Count,Avg
0,"Доставка и обработка возврата, отмены, невыкупа","Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.",-186.80,2,-93.400000
1,"Доставка и обработка возврата, отмены, невыкупа","Зарядное устройство для iPhone, iPad, AirPods. Адаптер 20W, кабель Lightning to USB Type-C, 1 метр белый. Быстрая зарядка.",-123.00,1,-123.000000
2,Доставка покупателю,"Держатель кухонный для разделочных досок, для сковород, тарелок, кастрюль, 3 см х 5 см х 5 см, 1 шт",1252.55,29,43.191379
3,Доставка покупателю,"Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.",1753.49,4,438.372500
4,Доставка покупателю,"Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.. Уцененный товар",425.43,1,425.430000
5,Доставка покупателю,"Зарядное устройство для iPhone, iPad, AirPods. Адаптер 20W, кабель Lightning to USB Type-C, 1 метр белый. Быстрая зарядка.",4441.09,10,444.109000
6,Доставка покупателю,Кабель Lightning to USB Type-C. Провод Lightning to USB Type-C. 1 метр белый.,229.95,1,229.950000
7,Доставка покупателю,Кабель USB Type-C to USB Type-C. Провод USB Type-C to USB Type-C. 1 метр белый.,945.22,10,94.522000
8,Доставка покупателю,"Органайзер для хранения вещей. 36 х 25 х 20 см, 7 отделений. 2 шт.",7330.71,14,523.622143
9,Доставка покупателю,"Электронные кухонные весы кухонныевесы, белый",554.19,2,277.095000


##### Income by product

In [156]:
# report_df\
report_df[
    (report_df['Тип начисления'] != 'Доставка покупателю') 
    | (report_df['За продажу или возврат до вычета комиссий и услуг'] > 100)
]\
    .replace(to_replace='^.*в формате "(.+?)" в период.*$', value=r'\1', regex=True)\
    .groupby(['Тип начисления', 'Название товара или услуги'], as_index=False, dropna=False)\
    .agg(Sum=('Итого', 'sum'), Count=('Количество', 'sum'))\
    .assign(Avg=lambda row: row['Sum'] / row['Count'])\
    .groupby(['Название товара или услуги'], as_index=False)\
    .agg(income_sum=('Sum', 'sum'), income_avg=('Avg', 'sum'))

,Название товара или услуги,income_sum,income_avg
0,"Держатель кухонный для разделочных досок, для сковород, тарелок, кастрюль, 3 см х 5 см х 5 см, 1 шт",1207.36,41.517676
1,"Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.",1553.47,342.769167
2,"Зарядное устройство для Sumsung, Xiaomi, Huawei, Honor. Адаптер 20W, кабель USB Type-C to USB Type-C, 1 метр белый. Быстрая зарядка.. Уцененный товар",420.12,420.120000
3,"Зарядное устройство для iPhone, iPad, AirPods. Адаптер 20W, кабель Lightning to USB Type-C, 1 метр белый. Быстрая зарядка.",4311.81,314.829000
4,Кабель Lightning to USB Type-C. Провод Lightning to USB Type-C. 1 метр белый.,224.88,229.225714
5,Кабель USB Type-C to USB Type-C. Провод USB Type-C to USB Type-C. 1 метр белый.,918.11,93.095158
6,"Органайзер для хранения вещей с мягкими стенками. 36 х 25 х 20 см, 9 отделений, серый.",-6.50,-3.250000
7,"Органайзер для хранения вещей. 36 х 25 х 20 см, 7 отделений.",-9.38,-0.938000
8,"Органайзер для хранения вещей. 36 х 25 х 20 см, 7 отделений. 2 шт.",7185.53,517.023052
9,Продвижение в поиске,-809.52,-inf
